### **Bài 12: Bài tập về nhà: Ứng dụng mô hình sequence2sequence cho bài toán sinh văn bản (text generation)**
Tổng quan: Ở bài tập này chúng ta sẽ ôn lại cách xây dựng và sử dụng mô hình seq2seq cho bài toán sinh văn bản.

**1. Chuẩn bị dữ liệu và tiền xử lý**

Trong bài tập này chúng ta sẽ xử lý dữ liệu của bài toán tóm tắt văn bản để thực nghiệm cho bài toán sinh văn bản. Trong bài toán tóm tắt văn bản, input của chương trình sẽ là 1 văn bản dài và output sẽ là 1 văn bản ngắn hơn và chứa những thông tin quan trọng của văn bản đầu vào. Ngược lại với bài toán trên, trong bài toán sinh văn bản, chúng ta muốn input đầu vào là 1 vài keyword hoặc 1 đoạn văn ngắn và output ra 1 đoạn văn dài. Vì thế, ta hoàn toàn có thể sử dụng dữ liệu trong bài toán tóm tắt văn bản để huấn luyện cho bài toán sinh văn bản, với input là câu đã được tóm tắt và output là đoạn văn gốc.

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

Bài tập 1: Bạn hãy download 1 bộ dữ liệu tóm tắt văn bản và tiền xử lý chúng. Có thể dùng 1 trong các bộ dữ liệu dưới đây
*   Bộ gigaword : https://drive.google.com/open?id=0B6N7tANPyVeBNmlSX19Ld2xDU1E
*   Bộ CNN/DM : https://s3.amazonaws.com/opennmt-models/Summary/cnndm.tar.gz

Yêu cầu : Sau khi tiền xử lý, chúng ta sẽ có 4 file data gồm :
*   train.input.txt : Chứa các câu tóm tắt dùng để huấn luyện mô hình, thường chiếm 80% kích thước tổng dữ liệu
*   train.output.txt : Chứa các đoạn văn bản gốc ứng với các tóm tắt.
*   valid.input.txt : Chứa các câu tóm tắt dùng để đánh giá mô hình, thường chiếm 10% kích thưởng tổng dữ liệu.
*   valid.output.txt : Chứa các đoạn văn bản gốc ứng vs các tóm tắt.

Lưu ý : Nếu bạn để max_length của dữ liệu quá lớn, thì mô hình của bạn sẽ rất to và có thể gây tràn RAM.






In [2]:
!wget https://s3.amazonaws.com/opennmt-models/Summary/cnndm.tar.gz

--2024-10-18 04:11:46--  https://s3.amazonaws.com/opennmt-models/Summary/cnndm.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.134.96, 16.182.73.112, 54.231.227.72, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.134.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500375629 (477M) [application/x-gzip]
Saving to: ‘cnndm.tar.gz’

cnndm.tar.gz        100%[===================>] 477.20M  52.9MB/s    in 9.5s    

2024-10-18 04:11:56 (50.2 MB/s) - ‘cnndm.tar.gz’ saved [500375629/500375629]



In [3]:
!tar -xvzf cnndm.tar.gz

test.txt.src
test.txt.tgt.tagged
train.txt.src
train.txt.tgt.tagged
val.txt.src
val.txt.tgt.tagged


In [4]:
# YOUR CODE HERE
def preprocess_sentence(sentence):
    sentence = "<start> "+sentence.strip()+" <end>"
    return sentence

def load_data(source_file, target_file, number_of_examples):
    max_len = 50
    source_sents = open(source_file, "r").readlines()
    target_sents = open(target_file, "r").readlines()
    assert len(source_sents) == len(target_sents)

    source_data, target_data = [], []
    for source_sentence, target_sentence in zip(source_sents[:number_of_examples],
                                                target_sents[:number_of_examples]):
        if len(source_sentence.strip().split()) > max_len or len(target_sentence.strip().split()) > max_len:
            continue
        source_data.append(preprocess_sentence(source_sentence))
        target_data.append(preprocess_sentence(target_sentence))

    return source_data, target_data

def tokenizer(sentences):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='')
    tokenizer.fit_on_texts(sentences)
    sent_tensors = tokenizer.texts_to_sequences(sentences)
    sent_tensors = tf.keras.preprocessing.sequence.pad_sequences(sent_tensors,
                                                            padding='post')

    return sent_tensors, tokenizer

def create_data(source_path, target_path, number_of_examples):
  # YOUR CODE HERE
    source_data, target_data = load_data(source_path, target_path, number_of_examples)
    source_tensors, source_tokenizer = tokenizer(source_data)
    target_tensors, target_tokenizer = tokenizer(target_data)
    return source_tensors, target_tensors, source_tokenizer, target_tokenizer

number_of_examples = -1
train_src_tensors, train_tgt_tensors, train_src_tokenizer, train_tgt_tokenizer = create_data("train.txt.src", "train.txt.tgt.tagged", number_of_examples)
# valid_src_tensors, valid_tgt_tensors, _, _ = create_data("val.txt.src", "val.txt.tgt.tagged", -1)

max_length_source, max_length_target = train_src_tensors.shape[1], train_tgt_tensors.shape[1]
# print(len(train_src_tensors), len(valid_src_tensors))

**2. Sử dụng tf.data.Dataset để tạo dữ liệu huấn luyện**

Bài tập 2: Hãy xem lại bài tập seq2seq cho bài toán dịch máy để thực hiện cách build data theo batch.

In [5]:
BUFFER_SIZE = len(train_src_tensors)
BATCH_SIZE = 64
vocab_src_size = len(train_src_tokenizer.word_index)+1
vocab_tgt_size = len(train_tgt_tokenizer.word_index)+1
steps_per_epoch = len(train_src_tensors)//BATCH_SIZE
# YOUR CODE HERE

train_dataset = tf.data.Dataset.from_tensor_slices((train_src_tensors, train_tgt_tensors))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 52]), TensorShape([64, 52]))

**3. Mô hình Seq2Seq với Attention**

Bài tập 3: Hãy viết lại các thành phần Encoder, Attention, Decoder theo cách hiểu của bạn. Ngoài BahdanauAttention, LuongAttention cũng rất phổ biến, bạn hãy thử implement LuongAttention.

In [6]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_state_size, batch_sz):
        # YOUR CODE HERE
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.hidden_state_size = hidden_state_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
        self.gru = tf.keras.layers.GRU(self.hidden_state_size,
                                        return_sequences=True,
                                        return_state=True,)

    def call(self, x, hidden):
        # YOUR CODE HERE
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        # YOUR CODE HERE
        return tf.zeros((self.batch_sz, self.hidden_state_size))

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_state_size, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.hidden_state_size = hidden_state_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
        self.gru = self.gru = tf.keras.layers.GRU(self.hidden_state_size,
                                                return_sequences=True,
                                                return_state=True,)


        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.hidden_state_size)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [7]:
embedding_dim = 512
hidden_state_size = 512

encoder = Encoder(vocab_src_size, embedding_dim, hidden_state_size, BATCH_SIZE)
attention_layer = BahdanauAttention(10)
decoder = Decoder(vocab_tgt_size, embedding_dim, hidden_state_size, BATCH_SIZE)

**4. Hàm tối ưu và hàm lỗi**

Hàm tối ưu Adam và hàm lỗi Cross Entropy được dùng rất phổ biến trong các mô hình học sâu, trong phần này, chúng ra sẽ dùng lại các hàm đó nhé.

In [8]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    loss_ = loss_object(real, pred)

    mask = tf.cast(real != 0, dtype=tf.float32)
    loss_ *= mask

    return tf.reduce_mean(loss_)

**5. Huấn luyện**

Giờ đã có đủ nguyên liệu và mô hình rồi, ta hãy cùng huấn luyện 1 mô hình có thể sinh văn bản.

In [9]:
@tf.function
def train_step(source, target, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(source, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([train_tgt_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

        for t in range(1, target.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(target[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(target[:, t], 1)

    batch_loss = (loss / int(target.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss


checkpoint_dir = './model_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    for (batch, (source, target)) in enumerate(train_dataset.take(steps_per_epoch)):
        batch_loss = train_step(source, target, enc_hidden)
        total_loss += batch_loss
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                    batch_loss.numpy()))
    # saving (checkpoint) the model every 1 epochs
    checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 5.5485
Epoch 1 Loss 5.5485
Time taken for 1 epoch 113.86390805244446 sec



**6. Sinh văn bản**

Hãy sử dụng mô hình vừa được huấn luyện để thực hiện sinh văn bản.

In [15]:
def evaluate(input_sentence):
  # YOUR CODE HERE
    attention_plot = np.zeros((max_length_target, max_length_source))
    source_sentence = preprocess_sentence(input_sentence)

    # Handle out-of-vocabulary (OOV) words by mapping unknown words to a special token
    inputs = []
    for word in source_sentence.lower().split(' '):
        if word in train_src_tokenizer.word_index:
            inputs.append(train_src_tokenizer.word_index[word])
        else:
            # Use the OOV token index if the word is not found in the vocabulary
            inputs.append(train_src_tokenizer.word_index.get('<oov>', 0))  # Assuming '<oov>' is your OOV token

    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_source,
                                                         padding='post')

    inputs = tf.convert_to_tensor(inputs)

    result = ''
  # YOUR CODE HERE
    # Initialize hidden state
    hidden = encoder.initialize_hidden_state()[:inputs.shape[0]]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([train_tgt_tokenizer.word_index['<start>']] * inputs.shape[0], 1)

    for t in range(max_length_target):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.squeeze(attention_weights)
        attention_plot[t] = attention_weights

        predicted_id = np.argmax(predictions)

        # Ensure the predicted_id exists in the tokenizer's index_word
        if predicted_id in train_tgt_tokenizer.index_word:
            result += ' ' + train_tgt_tokenizer.index_word[predicted_id]
        else:
            result += ' <unknown>'  # Handle cases where the predicted ID is not found

        # Pass the predicted_id to the decoder for the next time step
        dec_input = tf.expand_dims([predicted_id], 1)

    return result, source_sentence, attention_plot



input_sentence = "hollywood shores up support for ocean 's thirteen"
original_article = "hollywood is planning a new sequel to adventure flick `` ocean 's eleven , '' with star george clooney set to reprise his role as a charismatic thief in `` ocean 's thirteen , '' the entertainment press said wednesday ."
result, input_sentence, attention_plot = evaluate(input_sentence)
print('Input: %s' % (input_sentence))
print('Output : {}'.format(result))

Input: <start> hollywood shores up support for ocean 's thirteen <end>
Output :  <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t> . </t> <t>
